In [3]:
import visa
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
import csv as csv
import datetime

from datetime import datetime
from visa import constants as viCon

rm = visa.ResourceManager()

scope = rm.open_resource('USB0::0x0957::0x179B::MY51250109::0::INSTR')
scope.enable_event(viCon.EventType.service_request,viCon.VI_QUEUE)
scope.VI_ATTR_MAX_QUEUE_LENGTH = 1

In [4]:
testname = "scope_readout/nbiot_test0_mqtt_psm5min1.csv"
#testname = "scope_readout/nb_iot_coap_startup.csv"
TEST_NUMBER = 50
df = pd.DataFrame()

In [5]:
scope.clear()
scope.write("*CLS") #Clearing the trigger register. Important!
scope.write("*SRE 1") #Service requests on triggers
# The WORD format is the most accurate for Keysight Oscilloscopes. Data is transferred as signed 16bit integers in 2 bytes.
scope.write(":WAV:SOUR MATH")
scope.write(":WAV:FORMAT WORD")
scope.write(":WAV:BYTEORDER MSBF") # Set the byte order to Big-Endian (default for Infinium oscilloscopes)
scope.read_termination = None # Since we're transferring binary data we need to remove the newline read termination

i = 0

In [6]:
print("Test started " + str(datetime.now()))

start = time.perf_counter()
while( i < TEST_NUMBER):
    scope.clear()
    scope.write("*CLS") #Clearing the trigger register. Important!
    scope.write("*TRG")
    try:
        scope.wait_on_event(viCon.EventType.service_request,1000)

        data = [time.perf_counter() - start]
        wfm_array = scope.query_binary_values(":WAV:DATA?", 'h', is_big_endian = True, container = list)
        
        data.extend(wfm_array)
        df["meas"+str(i)] = data
        
        
        print("Current measurment: " + str(i))               
        
        i = i + 1
        
    except visa.VisaIOError: 
        try:
            data = [time.perf_counter() - start]
            wfm_array = scope.query_binary_values(":WAV:DATA?", 'h', is_big_endian = True, container = list)
            data.extend(wfm_array)
            df["meas"+str(i)] = data
        
            print("Current measurment: " + str(i))
            
            i = i + 1
        except visa.VisaIOError:
            pass
df.to_csv(testname)

Test started 2020-05-08 16:41:57.752328
Current measurment: 0
Current measurment: 1
Current measurment: 2
Current measurment: 3
Current measurment: 4
Current measurment: 5
Current measurment: 6
Current measurment: 7
Current measurment: 8
Current measurment: 9
Current measurment: 10
Current measurment: 11
Current measurment: 12
Current measurment: 13
Current measurment: 14
Current measurment: 15
Current measurment: 16
Current measurment: 17
Current measurment: 18
Current measurment: 19
Current measurment: 20
Current measurment: 21
Current measurment: 22
Current measurment: 23
Current measurment: 24
Current measurment: 25
Current measurment: 26
Current measurment: 27
Current measurment: 28
Current measurment: 29


VisaIOError: VI_ERROR_IO (-1073807298): Could not perform operation because of I/O error.

In [7]:
df.to_csv(testname)

In [ ]:
y_inc = float(scope.query(":WAV:YINC?"))
y_or = float(scope.query(":WAV:YOR?"))
y_ref = float(scope.query(":WAV:YREF?"))
x_inc = float(scope.query(":WAV:XINC?"))